# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-027c3e5169-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Sport Workflow Series ("Olympic explorative search") 

Consider the following exploratory information need:

> investigate the results, in terms of participations and competitions won, in the past editions of the olympics between France and Germany

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q159821`  | Olympic games | node      |
| `wd:Q10843402`| Swimmer       | node |
| `wd:Q2066131` | Athlete       | node |
| `wd:Q142`     | France        | node |
| `wd:Q183`     | Germany       | node |


Also consider


```
?a wdt:P106/wdt:P279 wd:Q2066131
```

is the BGP to retrieve all instances of **athlete**


```
?p wdt:P27 wd:Q142
```

is the BGP to retrieve all **citizens of France**

## Workload Goals

1. Identify the BGP for participating or winning some competition

2. Identify the BGP for different sports or types of competitions

3. Is there some French or German Athlete participating to any olympic event with double citizenship?

4. Compare number of participations and winning events across different dimensions
 
   4.1 In which specific sport France has more winners than Germany
   
c
   
   4.3 For each country , which are the atheletes with the highest number of participations


In [11]:
# start your workflow here

In [5]:
#this query retrieve and count all instances of athlete
queryString = """
SELECT COUNT(*)
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '929008')]


1

In [ ]:
#1. Identify the BGP for participating or winning some competition

In [6]:
#check if there is something between olimpic games and athletes
queryString = """
SELECT ?q  
WHERE { 

wd:Q2066131 ?q wd:Q159821

} 
LIMIT 1

"""

print("Results")
run_query(queryString)




Results
Empty


0

In [16]:
#check if there is something related to olimpic games
queryString = """


SELECT ?a ?q  
WHERE { 

?a ?q wd:Q159821 

} 
LIMIT 10

"""
print("Results")
run_query(queryString)



Results
[('a', 'http://www.wikidata.org/entity/Q16150839'), ('q', 'http://www.wikidata.org/prop/direct/P1344')]
[('a', 'http://www.wikidata.org/entity/Q4823714'), ('q', 'http://www.wikidata.org/prop/direct/P1344')]
[('a', 'http://www.wikidata.org/entity/Q60566409'), ('q', 'http://www.wikidata.org/prop/direct/P138')]
[('a', 'http://www.wikidata.org/entity/Q60566409'), ('q', 'http://www.wikidata.org/prop/direct/P144')]
[('a', 'http://www.wikidata.org/entity/Q8531'), ('q', 'http://www.wikidata.org/prop/direct/P179')]
[('a', 'http://www.wikidata.org/entity/Q8456'), ('q', 'http://www.wikidata.org/prop/direct/P179')]
[('a', 'http://www.wikidata.org/entity/Q5389'), ('q', 'http://www.wikidata.org/prop/direct/P2670')]
[('a', 'http://www.wikidata.org/entity/Q7215602'), ('q', 'http://www.wikidata.org/prop/direct/P301')]
[('a', 'http://www.wikidata.org/entity/Q8098'), ('q', 'http://www.wikidata.org/prop/direct/P31')]
[('a', 'http://www.wikidata.org/entity/Q8150'), ('q', 'http://www.wikidata.org/pr

10

In [17]:
#check if there is something related to athletes
queryString = """


SELECT ?a ?q  
WHERE { 


wd:Q2066131 ?a ?q 

} 
LIMIT 10

"""
print("Results")
run_query(queryString)



Results
[('a', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('q', 'http://wikiba.se/ontology#Item')]
[('a', 'http://www.wikidata.org/prop/direct-normalized/P1014'), ('q', 'http://vocab.getty.edu/aat/300236025')]
[('a', 'http://www.wikidata.org/prop/direct-normalized/P227'), ('q', 'https://d-nb.info/gnd/4056399-6')]
[('a', 'http://www.wikidata.org/prop/direct-normalized/P2347'), ('q', 'http://www.yso.fi/onto/yso/p3315')]
[('a', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('q', 'https://id.loc.gov/authorities/names/sh85009135')]
[('a', 'http://www.wikidata.org/prop/direct-normalized/P2581'), ('q', 'http://babelnet.org/rdf/s00006747n')]
[('a', 'http://www.wikidata.org/prop/direct-normalized/P268'), ('q', 'http://data.bnf.fr/ark:/12148/cb11933850z#about')]
[('a', 'http://www.wikidata.org/prop/direct-normalized/P268'), ('q', 'http://data.bnf.fr/ark:/12148/cb11965039b#about')]
[('a', 'http://www.wikidata.org/prop/direct-normalized/P3911'), ('q', 'http://zbw.eu/stw/descript

10

In [19]:
#this query retrieve 10 athletes
queryString = """
SELECT ?a
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131

} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q16046478')]
[('a', 'http://www.wikidata.org/entity/Q9459145')]
[('a', 'http://www.wikidata.org/entity/Q56368206')]
[('a', 'http://www.wikidata.org/entity/Q16016430')]
[('a', 'http://www.wikidata.org/entity/Q16215952')]
[('a', 'http://www.wikidata.org/entity/Q16218792')]
[('a', 'http://www.wikidata.org/entity/Q16232783')]
[('a', 'http://www.wikidata.org/entity/Q16233681')]
[('a', 'http://www.wikidata.org/entity/Q16236199')]
[('a', 'http://www.wikidata.org/entity/Q16236201')]


10

In [4]:
#this query to search relations with atletes 
queryString = """
SELECT DISTINCT ?q ?pname
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131.
?a ?q ?c.
?q <http://schema.org/name> ?pname
} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('q', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('q', 'http://www.wikidata.org/prop/direct/P1532'), ('pname', 'country for sport')]
[('q', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('q', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth')]
[('q', 'http://www.wikidata.org/prop/direct/P2003'), ('pname', 'Instagram username')]
[('q', 'http://www.wikidata.org/prop/direct/P21'), ('pname', 'sex or gender')]
[('q', 'http://www.wikidata.org/prop/direct/P2671'), ('pname', 'Google Knowledge Graph ID')]
[('q', 'http://www.wikidata.org/prop/direct/P27'), ('pname', 'country of citizenship')]
[('q', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('q', 'http://www.wikidata.org/prop/direct/P373'), ('pname', 'Commons category')]
[('q', 'http://www.wikidata.org/prop/direct/P569'), ('pname', 'date of birth')]
[('q', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport')]
[('q', 'http://www.wi

200

In [42]:
#the two BGP are: winning wdt:P2522/ partecipation wdt:P1344    

In [ ]:
#2. Identify the BGP for different sports or types of competitions

In [46]:

#this query to search relations with olimpiads
queryString = """
SELECT DISTINCT ?a ?pname
WHERE { 

wd:Q159821 ?a ?d.

?a <http://schema.org/name> ?pname
} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('a', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('a', 'http://www.wikidata.org/prop/direct/P214'), ('pname', 'VIAF ID')]
[('a', 'http://www.wikidata.org/prop/direct/P2257'), ('pname', 'event interval')]
[('a', 'http://www.wikidata.org/prop/direct/P227'), ('pname', 'GND ID')]
[('a', 'http://www.wikidata.org/prop/direct/P2347'), ('pname', 'YSO ID')]
[('a', 'http://www.wikidata.org/prop/direct/P244'), ('pname', 'Library of Congress authority ID')]
[('a', 'http://www.wikidata.org/prop/direct/P2581'), ('pname', 'BabelNet ID')]
[('a', 'http://www.wikidata.org/prop/direct/P269'), ('pname', 'IdRef ID')]
[('a', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('a', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('a', 'http://www.wikidata.org/prop/direct/P3417'), ('pname', 'Quora topic ID')]
[('a', 'http://www.wikidata.org/prop/dire

29

In [61]:
#this query to search relations with partecipation
queryString = """
SELECT DISTINCT ?d ?pname
WHERE { 

?a wdt:P1344 ?d.

?d <http://schema.org/name> ?pname
} 
LIMIT 10
"""

print("Results")
run_query(queryString)








Results
[('d', 'http://www.wikidata.org/entity/Q1053559'), ('pname', '2020 Winter Youth Olympics')]
[('d', 'http://www.wikidata.org/entity/Q1082132'), ('pname', '2004 UEFA European Under-17 Football Championship')]
[('d', 'http://www.wikidata.org/entity/Q1209347'), ('pname', '1987 Southeast Asian Games')]
[('d', 'http://www.wikidata.org/entity/Q1279521'), ('pname', '2013 Copa Libertadores')]
[('d', 'http://www.wikidata.org/entity/Q129304'), ('pname', '2013 CONCACAF U-20 Championship')]
[('d', 'http://www.wikidata.org/entity/Q162977'), ('pname', 'Egypt')]
[('d', 'http://www.wikidata.org/entity/Q1649567'), ('pname', '1965 Major League Baseball season')]
[('d', 'http://www.wikidata.org/entity/Q166551'), ('pname', '1997 FIFA Confederations Cup')]
[('d', 'http://www.wikidata.org/entity/Q216330'), ('pname', '24 Hours of Daytona')]
[('d', 'http://www.wikidata.org/entity/Q2365546'), ('pname', '2011 CONCACAF U-17 Championship')]


10

In [68]:
#now i search relations with Winter Youth Olympics

queryString = """
SELECT DISTINCT ?a ?pname
WHERE { 

wd:Q1053559 ?a ?d.

?a <http://schema.org/name> ?pname
} 
LIMIT 100
"""

print("Results")
run_query(queryString)









Results
[('a', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows')]
[('a', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by')]
[('a', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('a', 'http://www.wikidata.org/prop/direct/P2002'), ('pname', 'Twitter username')]
[('a', 'http://www.wikidata.org/prop/direct/P2003'), ('pname', 'Instagram username')]
[('a', 'http://www.wikidata.org/prop/direct/P2013'), ('pname', 'Facebook ID')]
[('a', 'http://www.wikidata.org/prop/direct/P2397'), ('pname', 'YouTube channel ID')]
[('a', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('a', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('a', 'http://www.wikidata.org/prop/direct/P3267'), ('pname', 'Flickr user ID')]
[('a', 'http://www.wikidata.org/prop/direct/P373'), ('pname', 'Commons category')]
[('a', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part')]
[('a', 'http://www.wikidata.org/prop/dir

23

In [70]:
#now i get different sports

queryString = """
SELECT DISTINCT ?a ?pname
WHERE { 

?a wdt:P641 ?d.

?d <http://schema.org/name> ?pname
} 
LIMIT 100
"""

print("Results")
run_query(queryString)




Results
[('a', 'http://www.wikidata.org/entity/Q81647791'), ('pname', '2020 Winter Youth Olympics')]
[('a', 'http://www.wikidata.org/entity/Q25059071'), ('pname', 'mahjong')]
[('a', 'http://www.wikidata.org/entity/Q3350839'), ('pname', 'mahjong')]
[('a', 'http://www.wikidata.org/entity/Q5412796'), ('pname', 'mahjong')]
[('a', 'http://www.wikidata.org/entity/Q65095582'), ('pname', 'mahjong')]
[('a', 'http://www.wikidata.org/entity/Q105064354'), ('pname', 'mahjong')]
[('a', 'http://www.wikidata.org/entity/Q105296554'), ('pname', 'mahjong')]
[('a', 'http://www.wikidata.org/entity/Q1814785'), ('pname', 'fives')]
[('a', 'http://www.wikidata.org/entity/Q16857179'), ('pname', 'fives')]
[('a', 'http://www.wikidata.org/entity/Q74119639'), ('pname', 'college soccer')]
[('a', 'http://www.wikidata.org/entity/Q105081584'), ('pname', 'college soccer')]
[('a', 'http://www.wikidata.org/entity/Q85798391'), ('pname', 'college soccer')]
[('a', 'http://www.wikidata.org/entity/Q74123524'), ('pname', 'colle

100

In [75]:
#now i get the things that have a relations with mahjong


queryString = """
SELECT DISTINCT ?a ?pname
WHERE { 

wd:Q25059071 ?d ?a.

?a <http://schema.org/name> ?pname
} 
LIMIT 100
"""

print("Results")
run_query(queryString)












Results
[('a', 'http://www.wikidata.org/entity/Q12263'), ('pname', 'mahjong')]
[('a', 'http://www.wikidata.org/entity/Q11422536'), ('pname', 'international sport governing body')]
[('a', 'http://www.wikidata.org/entity/Q2'), ('pname', 'Earth')]
[('a', 'http://www.wikidata.org/entity/Q807'), ('pname', 'Lausanne')]


4

In [76]:
#now i get the things that have a relations with mahjong sport


queryString = """
SELECT DISTINCT ?a ?pname
WHERE { 

wd:Q12263 ?d ?a.

?a <http://schema.org/name> ?pname
} 
LIMIT 100
"""

print("Results")
run_query(queryString)



Results
[('a', 'http://www.wikidata.org/entity/Q11416'), ('pname', 'gambling')]
[('a', 'http://www.wikidata.org/entity/Q131436'), ('pname', 'board game')]
[('a', 'http://www.wikidata.org/entity/Q2312410'), ('pname', 'sports discipline')]
[('a', 'http://www.wikidata.org/entity/Q29520'), ('pname', 'China')]
[('a', 'http://www.wikidata.org/entity/Q31629'), ('pname', 'type of sport')]
[('a', 'http://www.wikidata.org/entity/Q3244175'), ('pname', 'tabletop game')]
[('a', 'http://www.wikidata.org/entity/Q1272194'), ('pname', 'tile-based game')]
[('a', 'http://www.wikidata.org/entity/Q953673'), ('pname', 'Mahjong solitaire')]
[('a', 'http://www.wikidata.org/entity/Q105064360'), ('pname', 'Mahjongist')]
[('a', 'http://www.wikidata.org/entity/Q8595931'), ('pname', 'Category:Mahjong')]


10

In [ ]:
#in the previous query i obtained sport discipline and type of sport

In [ ]:
#3. Is there some French or German Athlete participating to any olympic event with double citizenship?

In [3]:
##this retrive all atletes of france and germany


queryString = """
SELECT DISTINCT ?a ?pname 
WHERE { 
{

  ?a wdt:P106/wdt:P279 wd:Q2066131.
  ?a wdt:P27 wd:Q142.
  
  ?a <http://schema.org/name> ?pname.

}
UNION
{

  ?a wdt:P106/wdt:P279 wd:Q2066131.
  ?a wdt:P27 wd:Q183.
  ?a <http://schema.org/name> ?pname

}

} 
LIMIT 100
"""

print("Results")
run_query(queryString)


Results
[('a', 'http://www.wikidata.org/entity/Q2935075'), ('pname', 'Camille Serme')]
[('a', 'http://www.wikidata.org/entity/Q908632'), ('pname', 'Grégory Gaultier')]
[('a', 'http://www.wikidata.org/entity/Q16745166'), ('pname', 'Christophe André')]
[('a', 'http://www.wikidata.org/entity/Q1067657'), ('pname', 'Thierry Lincou')]
[('a', 'http://www.wikidata.org/entity/Q3388824'), ('pname', 'Grégoire Marche')]
[('a', 'http://www.wikidata.org/entity/Q51115788'), ('pname', 'Vincent Droesbeke')]
[('a', 'http://www.wikidata.org/entity/Q3155042'), ('pname', 'Isabelle Stoehr')]
[('a', 'http://www.wikidata.org/entity/Q49725700'), ('pname', 'Charlotte Delsinne')]
[('a', 'http://www.wikidata.org/entity/Q104670987'), ('pname', 'Catherine Lebossé')]
[('a', 'http://www.wikidata.org/entity/Q15820890'), ('pname', 'Jean-Michel Arcucci')]
[('a', 'http://www.wikidata.org/entity/Q3460802'), ('pname', 'Mathieu Castagnet')]
[('a', 'http://www.wikidata.org/entity/Q3460814'), ('pname', 'Julien Balbo')]
[('a',

100

In [27]:
#i serach for relationship with partecipation
queryString = """
SELECT DISTINCT ?a ?pname 
WHERE { 


  ?q wdt:P1344 ?a.
  
  
  ?a <http://schema.org/name> ?pname.




} 
LIMIT 100
"""

print("Results")
run_query(queryString)


Results
[('a', 'http://www.wikidata.org/entity/Q1053559'), ('pname', '2020 Winter Youth Olympics')]
[('a', 'http://www.wikidata.org/entity/Q1082132'), ('pname', '2004 UEFA European Under-17 Football Championship')]
[('a', 'http://www.wikidata.org/entity/Q1209347'), ('pname', '1987 Southeast Asian Games')]
[('a', 'http://www.wikidata.org/entity/Q1279521'), ('pname', '2013 Copa Libertadores')]
[('a', 'http://www.wikidata.org/entity/Q129304'), ('pname', '2013 CONCACAF U-20 Championship')]
[('a', 'http://www.wikidata.org/entity/Q162977'), ('pname', 'Egypt')]
[('a', 'http://www.wikidata.org/entity/Q1649567'), ('pname', '1965 Major League Baseball season')]
[('a', 'http://www.wikidata.org/entity/Q166551'), ('pname', '1997 FIFA Confederations Cup')]
[('a', 'http://www.wikidata.org/entity/Q216330'), ('pname', '24 Hours of Daytona')]
[('a', 'http://www.wikidata.org/entity/Q2365546'), ('pname', '2011 CONCACAF U-17 Championship')]
[('a', 'http://www.wikidata.org/entity/Q244367'), ('pname', '2007–

100

In [29]:
#i serach for relationship with olimpic games
queryString = """
SELECT DISTINCT ?a ?pname 
WHERE { 


  wd:Q159821 ?q ?a.
  
  
  ?a <http://schema.org/name> ?pname.




} 
LIMIT 10
"""


print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q8098'), ('pname', '1904 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q8150'), ('pname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q8415'), ('pname', '1960 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q181278'), ('pname', '2020 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q18608583'), ('pname', 'recurring sporting event')]
[('a', 'http://www.wikidata.org/entity/Q40970'), ('pname', 'International Olympic Committee')]
[('a', 'http://www.wikidata.org/entity/Q5389'), ('pname', 'Olympic Games')]
[('a', 'http://www.wikidata.org/entity/Q8088'), ('pname', '1900 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q82414'), ('pname', 'Winter Olympic Games')]
[('a', 'http://www.wikidata.org/entity/Q8438'), ('pname', '1972 Summer Olympics')]


10

In [36]:
##this retrive all atletes of france and germany who participate in an olimpic event and show the event and the name


queryString = """
SELECT DISTINCT ?a ?pname ?dname
WHERE { 
{

  ?a wdt:P106/wdt:P279 wd:Q2066131.
  ?a wdt:P27 wd:Q142.
  
  
  #this get the olimpic events
  wd:Q159821 ?q ?d.
  #this get the partecipation 
  ?a wdt:P1344 ?d.
  
  
  ?d <http://schema.org/name> ?dname.
  ?a <http://schema.org/name> ?pname.

}
UNION
{

  ?a wdt:P106/wdt:P279 wd:Q2066131.
  ?a wdt:P27 wd:Q183.
  
  #this get the olimpic events
  wd:Q159821 ?q ?d.
  #this get the partecipation 
  ?a wdt:P1344 ?d.
  
  
  ?d <http://schema.org/name> ?dname.
  ?a <http://schema.org/name> ?pname

}

} 
LIMIT 10
"""

print("Results")
run_query(queryString)


Results
[('a', 'http://www.wikidata.org/entity/Q13639919'), ('pname', 'Joseph Goubert'), ('dname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q30133063'), ('pname', 'Paul Bronner'), ('dname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q5079280'), ('pname', 'Charles Imbault'), ('dname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q5373240'), ('pname', 'Emmanuel Gonat'), ('dname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q168971'), ('pname', 'Robert Dorgebray'), ('dname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q2356350'), ('pname', 'René Bougnol'), ('dname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q2357548'), ('pname', 'René Bondoux'), ('dname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q3092463'), ('pname', 'Félix Grimonprez'), ('dname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q2960736'), ('pname', 'Charles de

10

In [54]:
##this retrive all atletes of france and germany who participate in an olimpic event and have double nationality


queryString = """
SELECT DISTINCT ?a ?pname ?cname
WHERE { 

{

  ?a wdt:P106/wdt:P279 wd:Q2066131.
  ?a wdt:P27 wd:Q183.
  
  #this get the olimpic events
  wd:Q159821 ?q ?d.
  #this get the partecipation 
  ?a wdt:P1344 ?d.
  
  #this for being citizen of another state
  ?a wdt:P27 ?c.
  
  ?c <http://schema.org/name> ?cname.
  ?a <http://schema.org/name> ?pname.
  FILTER(?c!=wd:Q183)
}
UNION
{

  ?a wdt:P106/wdt:P279 wd:Q2066131.
  ?a wdt:P27 wd:Q142.
  
  
  #this get the olimpic events
  wd:Q159821 ?q ?d.
  #this get the partecipation 
  ?a wdt:P1344 ?d.
  
  #this for being citizen of another state
  ?a wdt:P27 ?c.
  
  ?c <http://schema.org/name> ?cname.
  ?a <http://schema.org/name> ?pname.
  FILTER(?c!=wd:Q142)
}
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q63257'), ('pname', 'Frank Kugler'), ('cname', 'United States of America')]
[('a', 'http://www.wikidata.org/entity/Q1995917'), ('pname', 'Robert Maysack'), ('cname', 'United States of America')]
[('a', 'http://www.wikidata.org/entity/Q15714305'), ('pname', 'Christian Deubler'), ('cname', 'United States of America')]
[('a', 'http://www.wikidata.org/entity/Q16623268'), ('pname', 'William Horschke'), ('cname', 'United States of America')]
[('a', 'http://www.wikidata.org/entity/Q1468179'), ('pname', 'Fritz Scheller'), ('cname', 'German Reich')]
[('a', 'http://www.wikidata.org/entity/Q5388931'), ('pname', 'Erik von Holst'), ('cname', 'Estonia')]
[('a', 'http://www.wikidata.org/entity/Q28501410'), ('pname', 'Erich Wichmann-Harbeck'), ('cname', 'Chile')]
[('a', 'http://www.wikidata.org/entity/Q215176'), ('pname', 'Paul de Bruyn'), ('cname', 'United States of America')]
[('a', 'http://www.wikidata.org/entity/Q69223'), ('pname', 'Friedrich Scherfke

10

In [56]:
##this ask if there are some French or German Athlete participating to any olympic event with double citizenship?


queryString = """

ASK WHERE{
SELECT DISTINCT ?a ?pname ?cname
WHERE { 

{

  ?a wdt:P106/wdt:P279 wd:Q2066131.
  ?a wdt:P27 wd:Q183.
  
  #this get the olimpic events
  wd:Q159821 ?q ?d.
  #this get the partecipation 
  ?a wdt:P1344 ?d.
  
  #this for being citizen of another state
  ?a wdt:P27 ?c.
  
  ?c <http://schema.org/name> ?cname.
  ?a <http://schema.org/name> ?pname.
  FILTER(?c!=wd:Q183)
}
UNION
{

  ?a wdt:P106/wdt:P279 wd:Q2066131.
  ?a wdt:P27 wd:Q142.
  
  
  #this get the olimpic events
  wd:Q159821 ?q ?d.
  #this get the partecipation 
  ?a wdt:P1344 ?d.
  
  #this for being citizen of another state
  ?a wdt:P27 ?c.
  
  ?c <http://schema.org/name> ?cname.
  ?a <http://schema.org/name> ?pname.
  FILTER(?c!=wd:Q142)
}
} 
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

In [ ]:
#4.1 in wich specific sport france has more winners than germany?

In [5]:


#select winners of france

queryString = """


SELECT DISTINCT ?a ?pname ?dname
WHERE { 



 #all citizens of france
 ?a wdt:P27 wd:Q142.
 #this get the winning
 ?a wdt:P2522 ?d.
 
 #this get the olimpic events
 wd:Q159821 ?q ?d.
 ?a <http://schema.org/name> ?pname.
 ?d <http://schema.org/name> ?dname.
}

"""

print("Results")
run_query(queryString)


Results
[('a', 'http://www.wikidata.org/entity/Q1150190'), ('pname', 'Cédric Burdet'), ('dname', '2008 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q1097044'), ('pname', 'Claude Onesta'), ('dname', '2008 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q1150266'), ('pname', 'Cédric Paty'), ('dname', '2008 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q1165461'), ('pname', 'Daouda Sow'), ('dname', '2008 Summer Olympics')]


4

In [28]:
#select sport and winners of france and respective sport

queryString = """


SELECT DISTINCT  ?a ?aname  ?ename ?dname
WHERE { 
 #all citizens of france
 ?a wdt:P27 wd:Q142.
 
 #this get me the sports
 ?d ?q wd:Q2312410.
 #this get me the people related to that sport
 ?a ?c  ?d.
 #this get me the winning 
 ?a wdt:P2522 ?e.
 
 ?a <http://schema.org/name> ?aname.
 ?e <http://schema.org/name> ?ename.
 ?d <http://schema.org/name> ?dname.
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q274636'), ('aname', 'Christophe Lemaitre'), ('ename', "2010 European Athletics Championships – men's 200 metres"), ('dname', 'sprinter')]
[('a', 'http://www.wikidata.org/entity/Q274636'), ('aname', 'Christophe Lemaitre'), ('ename', '2013 European Team Championships'), ('dname', 'sprinter')]
[('a', 'http://www.wikidata.org/entity/Q274636'), ('aname', 'Christophe Lemaitre'), ('ename', "2010 European Athletics Championships – men's 100 metres"), ('dname', 'sprinter')]
[('a', 'http://www.wikidata.org/entity/Q274636'), ('aname', 'Christophe Lemaitre'), ('ename', '2015 European Team Championships'), ('dname', 'sprinter')]
[('a', 'http://www.wikidata.org/entity/Q274636'), ('aname', 'Christophe Lemaitre'), ('ename', '2009 European Athletics Junior Championships'), ('dname', 'sprinter')]
[('a', 'http://www.wikidata.org/entity/Q274636'), ('aname', 'Christophe Lemaitre'), ('ename', '2012 European Athletics Championships'), ('dname', 'sprinter')]
[('

10

In [215]:
#select the sport winners of france and germany and respective sport

queryString = """


SELECT (COUNT(?a) AS ?winners)  
WHERE { 
{
 #all citizens of germany
 ?a wdt:P27 wd:Q183. 
 
 #this get me the sports
 ?d ?q wd:Q2312410.
 #this get me the people related to that sport
 ?a ?c  ?d.
 #this get me the winners 
 ?a wdt:P2522 ?e.
 
 ?a <http://schema.org/name> ?aname.

 ?d <http://schema.org/name> ?dname.
}

UNION

{
 #all citizens of france
 ?a wdt:P27 wd:Q142.
 
 #this get me the sports
 ?d ?q wd:Q2312410.
 #this get me the people related to that sport
 ?a ?c ?d.
 #this get me the winning 
 ?a wdt:P2522 ?e.
 
 ?a <http://schema.org/name> ?aname.

 ?d <http://schema.org/name> ?dname.
}


}GROUP BY ?d
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('winners', '39')]
[('winners', '71')]
[('winners', '2')]
[('winners', '1')]
[('winners', '11')]
[('winners', '4')]
[('winners', '2')]
[('winners', '2')]
[('winners', '6')]
[('winners', '13')]


10

In [220]:
#i search for the relationship between people and sport

queryString = """


SELECT DISTINCT  ?person ?personname  ?sport ?sportname ?relationname ?cname
WHERE { 
 #all citizens of france
 ?person wdt:P27 wd:Q142.
 
 #this get me the sports
 ?sport ?relation wd:Q2312410.
 #this get me the people related to that sport
 ?person ?c  ?sport.
 
 
 ?person <http://schema.org/name> ?personname.
 ?sport <http://schema.org/name> ?sportname.
 ?relation <http://schema.org/name> ?relationname.
  ?c <http://schema.org/name> ?cname.
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q1379905'), ('personname', 'Roger Bambuck'), ('sport', 'http://www.wikidata.org/entity/Q4009406'), ('sportname', 'sprinter'), ('relationname', 'instance of'), ('cname', 'occupation')]
[('person', 'http://www.wikidata.org/entity/Q273422'), ('personname', 'Nicole Duclos'), ('sport', 'http://www.wikidata.org/entity/Q334734'), ('sportname', '400 metres'), ('relationname', 'instance of'), ('cname', 'field of work')]
[('person', 'http://www.wikidata.org/entity/Q273422'), ('personname', 'Nicole Duclos'), ('sport', 'http://www.wikidata.org/entity/Q230057'), ('sportname', '4 × 400 metres relay'), ('relationname', 'instance of'), ('cname', 'field of work')]
[('person', 'http://www.wikidata.org/entity/Q273422'), ('personname', 'Nicole Duclos'), ('sport', 'http://www.wikidata.org/entity/Q4009406'), ('sportname', 'sprinter'), ('relationname', 'instance of'), ('cname', 'occupation')]
[('person', 'http://www.wikidata.org/entity/Q453754'), ('personna

10

In [328]:
#in wich specific sport france has more winners than germany?
#NOT RIGTH LOOK THE NEXT

queryString = """

SELECT ?sport ?winnersgermany ?winnersfrance
{



{
{
SELECT (COUNT(?person) AS ?winnersgermany) ?sport  
{ 

 #all citizens of germany
 ?person wdt:P27 wd:Q183. 
 
 #this get me the sports
 ?sport ?q wd:Q2312410.
 #this get me the people related to that sport
 ?person ?c  ?sport.
 #this get me the winners 
 ?person wdt:P2522 ?e.

}GROUP BY ?sport
}



{
SELECT (COUNT(?person) AS ?winnersfrance) ?sport
WHERE{
 #all citizens of france
 ?person wdt:P27 wd:Q142.
 
 #this get me the sports
 ?sport ?m wd:Q2312410.
 #this get me the people related to that sport
 ?person ?n ?sport.
 #this get me the winning 
 ?person wdt:P2522 ?l.
 

}GROUP BY ?sport
}
}
UNION
{


 SELECT (COUNT(?personG) AS ?winnersgermany) ?sport  (COUNT(?personF) AS ?winnersfrance) 
{ 


OPTIONAL{ #all citizens of germany
 ?personG wdt:P27 wd:Q183. 
 
 #this get me the sports
 #?sport ?q wd:Q2312410.
 #this get me the people related to that sport
 ?personG ?c  ?sport.
 #this get me the winners 
 ?personG wdt:P2522 ?e.
}

#all citizens of france
 ?personF wdt:P27 wd:Q142.
 
 #this get me the sports
 ?sport ?g wd:Q2312410.
 #this get me the people related to that sport
 ?personF ?d ?sport.
 #this get me the winning 
 ?personF wdt:P2522 ?f.

}GROUP BY ?sport



}






}GROUP BY ?sport


"""





print("Results")
run_query(queryString)




#NOT RIGTH LOOK THE NEXT


Results
[('sport', 'http://www.wikidata.org/entity/Q1026754'), ('winnersgermany', '3'), ('winnersfrance', '9')]
[('sport', 'http://www.wikidata.org/entity/Q926191'), ('winnersgermany', '0'), ('winnersfrance', '65')]
[('sport', 'http://www.wikidata.org/entity/Q9352234'), ('winnersgermany', '0'), ('winnersfrance', '1')]
[('sport', 'http://www.wikidata.org/entity/Q4179462'), ('winnersgermany', '0'), ('winnersfrance', '14')]
[('sport', 'http://www.wikidata.org/entity/Q240356'), ('winnersgermany', '0'), ('winnersfrance', '11')]
[('sport', 'http://www.wikidata.org/entity/Q9130691'), ('winnersgermany', '0'), ('winnersfrance', '1')]
[('sport', 'http://www.wikidata.org/entity/Q537769'), ('winnersgermany', '3'), ('winnersfrance', '8')]
[('sport', 'http://www.wikidata.org/entity/Q164761'), ('winnersgermany', '2'), ('winnersfrance', '11')]
[('sport', 'http://www.wikidata.org/entity/Q5386'), ('winnersgermany', '23'), ('winnersfrance', '28')]
[('sport', 'http://www.wikidata.org/entity/Q537769'), ('w

62

In [323]:
#in wich specific sport france has more winners than germany try to get?
#NOT RIGTH LOOK THE NEXT

queryString = """




 
SELECT (COUNT(?personG) AS ?winnersgermany) ?sport  (COUNT(?personF) AS ?winnersfrance) 
WHERE{ 

OPTIONAL{ #all citizens of germany
 ?personG wdt:P27 wd:Q183. 
 
 #this get me the sports
 #?sport ?q wd:Q2312410.
 #this get me the people related to that sport
 ?personG ?c  ?sport.
 #this get me the winners 
 ?personG wdt:P2522 ?e.
}


#all citizens of france
 ?personF wdt:P27 wd:Q142.
 
 #this get me the sports
 ?sport ?g wd:Q2312410.
 #this get me the people related to that sport
 ?personF ?d ?sport.
 #this get me the winning 
 ?personF wdt:P2522 ?f.

}GROUP BY ?sport






"""





print("Results")
run_query(queryString)
#NOT RIGTH LOOK THE NEXT


Results
[('winnersgermany', '0'), ('sport', 'http://www.wikidata.org/entity/Q10980'), ('winnersfrance', '1')]
[('winnersgermany', '308'), ('sport', 'http://www.wikidata.org/entity/Q4305887'), ('winnersfrance', '308')]
[('winnersgermany', '390'), ('sport', 'http://www.wikidata.org/entity/Q926191'), ('winnersfrance', '390')]
[('winnersgermany', '0'), ('sport', 'http://www.wikidata.org/entity/Q9352234'), ('winnersfrance', '1')]
[('winnersgermany', '0'), ('sport', 'http://www.wikidata.org/entity/Q240500'), ('winnersfrance', '2')]
[('winnersgermany', '24'), ('sport', 'http://www.wikidata.org/entity/Q537769'), ('winnersfrance', '24')]
[('winnersgermany', '3'), ('sport', 'http://www.wikidata.org/entity/Q917562'), ('winnersfrance', '3')]
[('winnersgermany', '0'), ('sport', 'http://www.wikidata.org/entity/Q335638'), ('winnersfrance', '2')]
[('winnersgermany', '0'), ('sport', 'http://www.wikidata.org/entity/Q5849'), ('winnersfrance', '6')]
[('winnersgermany', '0'), ('sport', 'http://www.wikidata

42

In [349]:
#in wich specific sport france has more winners than germany?
#I select the name and URI of the sport and the winners of france and germany or only france if germany has zero winners

#LOOK THIS ONE
queryString = """
SELECT ?sport ?winnersfrance ?winnersgermany ?sportname
WHERE{
{
SELECT ?sport ?winnersfrance ?winnersgermany
{
{ 
SELECT ?sport  (COUNT(?personF) AS ?winnersfrance) 
WHERE{ 




#all citizens of france
 ?personF wdt:P27 wd:Q142.
 
 #this get me the sports
 ?sport ?g wd:Q2312410.
 #this get me the people related to that sport
 ?personF ?d ?sport.
 #this get me the winning 
 ?personF wdt:P2522 ?f.

}GROUP BY ?sport

}


{
SELECT (COUNT(?personG) AS ?winnersgermany) ?sport   
{ 

 #all citizens of germany
 ?personG wdt:P27 wd:Q183. 
 
 #this get me the sports
 ?sport ?q wd:Q2312410.
 #this get me the people related to that sport
 ?personG ?c  ?sport.
 #this get me the winners 
 ?personG wdt:P2522 ?e.
}GROUP BY ?sport
}




FILTER(?winnersfrance>?winnersgermany)
}
}
UNION
{
SELECT ?sport  (COUNT(?personF) AS ?winnersfrance)
{
  #all citizens of france
 ?personF wdt:P27 wd:Q142.
 
 #this get me the sports
 ?sport ?g wd:Q2312410.
 #this get me the people related to that sport
 ?personF ?d ?sport.
 #this get me the winning 
 ?personF wdt:P2522 ?f.
  FILTER NOT EXISTS {
        #all citizens of germany
         ?personG wdt:P27 wd:Q183. 
 
         #this get me the sports
         ?sport ?q wd:Q2312410.
         #this get me the people related to that sport
         ?personG ?c  ?sport.
         #this get me the winners 
         ?personG wdt:P2522 ?e.  
        }


}

}
?sport <http://schema.org/name> ?sportname.
}







"""





print("Results")
run_query(queryString)



Results
[('sport', 'http://www.wikidata.org/entity/Q1026754'), ('winnersfrance', '9'), ('winnersgermany', '3'), ('sportname', '60 metres hurdles')]
[('sport', 'http://www.wikidata.org/entity/Q4179462'), ('winnersfrance', '14'), ('winnersgermany', '7'), ('sportname', "ladies' singles")]
[('sport', 'http://www.wikidata.org/entity/Q164761'), ('winnersfrance', '11'), ('winnersgermany', '2'), ('sportname', '100 metres')]
[('sport', 'http://www.wikidata.org/entity/Q211155'), ('winnersfrance', '11'), ('winnersgermany', '2'), ('sportname', '200 metres')]
[('sport', 'http://www.wikidata.org/entity/Q246681'), ('winnersfrance', '11'), ('winnersgermany', '2'), ('sportname', '60 metres')]
[('sport', 'http://www.wikidata.org/entity/Q230061'), ('winnersfrance', '12'), ('winnersgermany', '5'), ('sportname', '4 × 100 metres relay')]
[('sport', 'http://www.wikidata.org/entity/Q165704'), ('winnersfrance', '4'), ('winnersgermany', '1'), ('sportname', 'high jump')]
[('sport', 'http://www.wikidata.org/entit

37

In [ ]:
#4.2 which country has the most female winners 

In [44]:

#first i get all the relationship with winners

queryString = """


SELECT  ?people  ?aname ?dname 
WHERE { 

 #all citizens of germany
 #?people wdt:P27 ?state. 
 
 #this get me the winners 
 ?people wdt:P2522 ?victory.
 
 ?people ?a ?d.
  
 ?a <http://schema.org/name> ?aname.
 ?d <http://schema.org/name> ?dname.

}
LIMIT 10
"""

print("Results")
run_query(queryString)



Results
[('people', 'http://www.wikidata.org/entity/Q22062004'), ('aname', 'follows'), ('dname', 'Verandas Willems 2015')]
[('people', 'http://www.wikidata.org/entity/Q22062004'), ('aname', 'follows'), ('dname', 'Verandas Willems 2015')]
[('people', 'http://www.wikidata.org/entity/Q22062004'), ('aname', 'follows'), ('dname', 'Verandas Willems 2015')]
[('people', 'http://www.wikidata.org/entity/Q22062004'), ('aname', 'general manager'), ('dname', 'Ivan De Schamphelaere')]
[('people', 'http://www.wikidata.org/entity/Q22062004'), ('aname', 'general manager'), ('dname', 'Ivan De Schamphelaere')]
[('people', 'http://www.wikidata.org/entity/Q22062004'), ('aname', 'general manager'), ('dname', 'Ivan De Schamphelaere')]
[('people', 'http://www.wikidata.org/entity/Q16236242'), ('aname', 'father'), ('dname', 'Péter R. Kárpáti')]
[('people', 'http://www.wikidata.org/entity/Q16201535'), ('aname', 'participant in'), ('dname', "RuPaul's Drag Race, season 4")]
[('people', 'http://www.wikidata.org/ent

10

In [47]:
#i get all the relationship whit miss romania to get female

queryString = """


SELECT   ?a ?aname ?d ?dname 
WHERE { 

 wd:Q16231846 ?a ?d.
  
 ?a <http://schema.org/name> ?aname.
 ?d <http://schema.org/name> ?dname.

}
LIMIT 100
"""

print("Results")
run_query(queryString)




Results
[('a', 'http://www.wikidata.org/prop/direct/P2522'), ('aname', 'victory'), ('d', 'http://www.wikidata.org/entity/Q498509'), ('dname', 'Miss Romania')]
[('a', 'http://www.wikidata.org/prop/direct/P735'), ('aname', 'given name'), ('d', 'http://www.wikidata.org/entity/Q22666945'), ('dname', 'Oana')]
[('a', 'http://www.wikidata.org/prop/direct/P106'), ('aname', 'occupation'), ('d', 'http://www.wikidata.org/entity/Q18581305'), ('dname', 'beauty pageant contestant')]
[('a', 'http://www.wikidata.org/prop/direct/P19'), ('aname', 'place of birth'), ('d', 'http://www.wikidata.org/entity/Q188673'), ('dname', 'Târgoviște')]
[('a', 'http://www.wikidata.org/prop/direct/P31'), ('aname', 'instance of'), ('d', 'http://www.wikidata.org/entity/Q5'), ('dname', 'human')]
[('a', 'http://www.wikidata.org/prop/direct/P21'), ('aname', 'sex or gender'), ('d', 'http://www.wikidata.org/entity/Q6581072'), ('dname', 'female')]
[('a', 'http://www.wikidata.org/prop/direct/P734'), ('aname', 'family name'), ('d

7

In [ ]:
#now i got female wich is Q6581072 and also  sex or gender relation P21

In [54]:
#
#i order the countries whith the most female winners to get the max
queryString = """


SELECT  ?nationality (COUNT(?person) AS ?winners)
WHERE { 
#they are female
?person wdt:P21 wd:Q6581072.

 #this get me the winners 
 ?person wdt:P2522 ?e.
 
 #this get me the nationality
 ?person wdt:P27 ?nationality.
 


}GROUP BY ?nationality
 ORDER BY DESC (?winners)



LIMIT 1000
"""

print("Results")
run_query(queryString)


Results
[('nationality', 'http://www.wikidata.org/entity/Q30'), ('winners', '644')]
[('nationality', 'http://www.wikidata.org/entity/Q155'), ('winners', '251')]
[('nationality', 'http://www.wikidata.org/entity/Q159'), ('winners', '230')]
[('nationality', 'http://www.wikidata.org/entity/Q142'), ('winners', '190')]
[('nationality', 'http://www.wikidata.org/entity/Q145'), ('winners', '177')]
[('nationality', 'http://www.wikidata.org/entity/Q17'), ('winners', '164')]
[('nationality', 'http://www.wikidata.org/entity/Q801'), ('winners', '144')]
[('nationality', 'http://www.wikidata.org/entity/Q38'), ('winners', '139')]
[('nationality', 'http://www.wikidata.org/entity/Q717'), ('winners', '137')]
[('nationality', 'http://www.wikidata.org/entity/Q29'), ('winners', '120')]
[('nationality', 'http://www.wikidata.org/entity/Q16'), ('winners', '116')]
[('nationality', 'http://www.wikidata.org/entity/Q183'), ('winners', '111')]
[('nationality', 'http://www.wikidata.org/entity/Q928'), ('winners', '95'

147

In [10]:
#LOOK THE NEXT
#i get the female winners of FRANCE

queryString = """




SELECT (COUNT(?person) AS ?winners)
WHERE { 
 #they are female
 ?person wdt:P21 wd:Q6581072.


 
 #all citizens of france
 ?person wdt:P27 wd:Q142.
 #this get the winning
 ?person wdt:P2522 ?d.
 
 #this get the olimpic events
 



}



LIMIT 1000
"""

print("Results")
run_query(queryString)

Results
[('winners', '190')]


1

In [22]:
#THIS query retrives all the number of female athletes winners for FRANCE
queryString = """
SELECT (COUNT(?personF) AS ?winnersFrance) {
SELECT DISTINCT  ?personF
WHERE{ 


#his get only the female
 ?personF wdt:P21 wd:Q6581072.
#all citizens of France
 ?personF wdt:P27 wd:Q142.
 
 #this get me the sports
 ?sport ?g wd:Q2312410.
 #this get me the people related to that sport
 ?personF ?d ?sport.
 #this get me the winning 
 ?personF wdt:P2522 ?f.

}
}



"""

print("Results")
run_query(queryString)


Results
[('winnersFrance', '24')]


1

In [20]:
#THIS query retrives all the number of female athlets winners for GERMANY
queryString = """
SELECT (COUNT(?personF) AS ?winnersGermany) {
SELECT DISTINCT  ?personF
WHERE{ 


#his get only the female
 ?personF wdt:P21 wd:Q6581072.
#all citizens of Germany
 ?personF wdt:P27 wd:Q183.
 
 #this get me the sports
 ?sport ?g wd:Q2312410.
 #this get me the people related to that sport
 ?personF ?d ?sport.
 #this get me the winning 
 ?personF wdt:P2522 ?f.

}
}



"""

print("Results")
run_query(queryString)



Results
[('winnersGermany', '22')]


1

In [ ]:
#so France has more female athletes winner than germany

In [97]:
#4.3 For each country , which are the atheletes with the highest number of participations


#i get the number of partecipation for each atehlete
queryString = """


SELECT  ?person (COUNT(?partecipation) AS ?numberpartecipation)
WHERE { 


 #this get me the partecipation
 ?person wdt:P1344 ?partecipation.
 
 #this means they are atheletes
 ?person wdt:P106/wdt:P279 wd:Q2066131.
 
 #this get me the nationality
 ?person wdt:P27 ?nationality.
 

}GROUP BY ?person
 



LIMIT 10
"""

print("Results")
run_query(queryString)








Results
[('person', 'http://www.wikidata.org/entity/Q65928813'), ('numberpartecipation', '4')]
[('person', 'http://www.wikidata.org/entity/Q27780740'), ('numberpartecipation', '3')]
[('person', 'http://www.wikidata.org/entity/Q30112944'), ('numberpartecipation', '2')]
[('person', 'http://www.wikidata.org/entity/Q3353111'), ('numberpartecipation', '2')]
[('person', 'http://www.wikidata.org/entity/Q1441312'), ('numberpartecipation', '1')]
[('person', 'http://www.wikidata.org/entity/Q27906295'), ('numberpartecipation', '1')]
[('person', 'http://www.wikidata.org/entity/Q28105992'), ('numberpartecipation', '2')]
[('person', 'http://www.wikidata.org/entity/Q27913911'), ('numberpartecipation', '4')]
[('person', 'http://www.wikidata.org/entity/Q56253560'), ('numberpartecipation', '1')]
[('person', 'http://www.wikidata.org/entity/Q6491794'), ('numberpartecipation', '1')]


10

In [366]:


#get the number of partecipations and countries
queryString = """



SELECT  ?nationality (MAX(?numberpartecipation) AS ?partecipations) 
{

SELECT  ?person ?nationality (COUNT(?partecipation) AS ?numberpartecipation) 
WHERE { 


 #this get me the partecipation
 ?person wdt:P1344 ?partecipation.
 
 #this means they are atheletes
 ?person wdt:P106/wdt:P279 wd:Q2066131.
 
 #this get me the nationality
 ?person wdt:P27 ?nationality.
 

}GROUP BY ?person ?nationality
 


}GROUP BY ?nationality 


"""

print("Results")
run_query(queryString)









Results
[('nationality', 'http://www.wikidata.org/entity/Q114'), ('partecipations', '34')]
[('nationality', 'http://www.wikidata.org/entity/Q889'), ('partecipations', '6')]
[('nationality', 'http://www.wikidata.org/entity/Q34'), ('partecipations', '109')]
[('nationality', 'http://www.wikidata.org/entity/Q817'), ('partecipations', '18')]
[('nationality', 'http://www.wikidata.org/entity/Q28'), ('partecipations', '52')]
[('nationality', 'http://www.wikidata.org/entity/Q1009'), ('partecipations', '14')]
[('nationality', 'http://www.wikidata.org/entity/Q36'), ('partecipations', '122')]
[('nationality', 'http://www.wikidata.org/entity/Q36704'), ('partecipations', '24')]
[('nationality', 'http://www.wikidata.org/entity/Q77'), ('partecipations', '19')]
[('nationality', 'http://www.wikidata.org/entity/Q1007'), ('partecipations', '7')]
[('nationality', 'http://www.wikidata.org/entity/Q225'), ('partecipations', '13')]
[('nationality', 'http://www.wikidata.org/entity/Q23635'), ('partecipations', '

333

In [4]:
#4.3 For each country , which are the atheletes with the highest number of participations I select multiple athletes if there are multiple
queryString = """

SELECT ?person ?nationality ?partecipations 
WHERE{
{
SELECT  ?person ?nationality (COUNT(?partecipation) AS ?partecipations) 
WHERE { 


 #this get me the partecipation
 ?person wdt:P1344 ?partecipation.
 
 #this means they are atheletes
 ?person wdt:P106/wdt:P279 wd:Q2066131.
 
 #this get me the nationality
 ?person wdt:P27 ?nationality.
 

}GROUP BY ?person ?nationality
}

#FILTER(?numberpartecipation=?partecipations)
{
SELECT  ?nationality (MAX(?numberpartecipation) AS ?partecipations) 
{


{
SELECT  ?person ?nationality (COUNT(?partecipation) AS ?numberpartecipation) 
WHERE { 


 #this get me the partecipation
 ?person wdt:P1344 ?partecipation.
 
 #this means they are atheletes
 ?person wdt:P106/wdt:P279 wd:Q2066131.
 
 #this get me the nationality
 ?person wdt:P27 ?nationality.
 

}GROUP BY ?person ?nationality
} 


}GROUP BY ?nationality 
}

}
ORDER BY DESC(?nationality)


LIMIT 1000
"""

print("Results")
run_query(queryString)






Results
[('person', 'http://www.wikidata.org/entity/Q19559926'), ('nationality', 'nodeID://b174387'), ('partecipations', '1')]
[('person', 'http://www.wikidata.org/entity/Q545055'), ('nationality', 'http://www.wikidata.org/entity/Q986'), ('partecipations', '29')]
[('person', 'http://www.wikidata.org/entity/Q2600792'), ('nationality', 'http://www.wikidata.org/entity/Q983'), ('partecipations', '5')]
[('person', 'http://www.wikidata.org/entity/Q17432619'), ('nationality', 'http://www.wikidata.org/entity/Q977'), ('partecipations', '2')]
[('person', 'http://www.wikidata.org/entity/Q2370576'), ('nationality', 'http://www.wikidata.org/entity/Q977'), ('partecipations', '2')]
[('person', 'http://www.wikidata.org/entity/Q21474069'), ('nationality', 'http://www.wikidata.org/entity/Q977'), ('partecipations', '2')]
[('person', 'http://www.wikidata.org/entity/Q792772'), ('nationality', 'http://www.wikidata.org/entity/Q977'), ('partecipations', '2')]
[('person', 'http://www.wikidata.org/entity/Q17735

441